In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import json
import os

current_dir = os.getcwd()

# JSON 파일 경로
json_file_path = os.path.join(current_dir, 'products.json')

# JSON 파일을 읽어와서 파싱
with open(json_file_path, 'r', encoding='utf-8') as file:
    products = json.load(file)

# 상품 정보 출력
for product in products:
    print(f"상품: {product['product_name']}, 기본 판매 수량: {product['quantity']}개, 기본 판매 수량의 가격: {product['price']}원")

상품: 떡케익5호, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 54000원
상품: 무지개 백설기 케익, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 51500원
상품: 미니 백설기, 기본 판매 수량: 35개, 기본 판매 수량의 가격: 31500원
상품: 개별 모듬팩, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 13500원


In [3]:
from langchain_core.prompts import ChatPromptTemplate

request_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문 관련 요청을 분류하는 로봇이야.
            주문 관련 요청을 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'
            """
        ),
        ("human", "{input}"),

    ]
)

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [5]:
from langchain_core.runnables import RunnablePassthrough

classify_request_chain = RunnablePassthrough.assign(request_type=request_type_prompt | model)

order_id 어케 얻음?

In [4]:
from pydantic import BaseModel, Field, conint, condecimal
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class OrderItemData(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    quantity: conint(gt=0) = Field(..., description="Quantity of the product")
    price: condecimal(max_digits=10, decimal_places=2) = Field(..., description="Price of the product")

class CreateOrderData(BaseModel):
    user_id: int = Field(..., description="ID of the user creating the order")
    items: List[OrderItemData] = Field(..., description="List of items to order")

class UpdateOrderStatusData(BaseModel):
    order_id: int = Field(..., description="ID of the order to update")
    new_status: str = Field(..., description="New status of the order")

create_order_parser = PydanticOutputParser(pydantic_object=CreateOrderData)
update_order_parser = PydanticOutputParser(pydantic_object=UpdateOrderStatusData)

PromptTemplate 역시 Chat model prompt로 사용 가능?

In [10]:
template = """
너는 각 주문 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.
답변에는 추출한 인자만 포함시켜.

상품 정보
{products}

user_id: {user_id}


user input: {input}
"""

from langchain_core.prompts import PromptTemplate

extract_order_args_prompt = PromptTemplate(
    template=template,
    input_variables=["uesr_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions()]},
)

In [11]:
extract_order_args_chain = extract_order_args_prompt | model
extract_order_args_chain

PromptTemplate(input_variables=['input', 'products', 'user_id'], partial_variables={'format_instructions': ['The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"OrderItemData": {"properties": {"product_name": {"description": "Name of the product", "title": "Product Name", "type": "string"}, "quantity": {"description": "Quantity of the product", "exclusiveMinimum": 0, "title": "Quantity", "type": "integer"}, "price": {"anyOf": [{"type": "number"}, {"type": "string"}], "description": "Price of the product", "title": "Price"}}, "required": ["product_name", "quantity", "price"], 

In [12]:
response = extract_order_args_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
response

AIMessage(content='상품명: 개별 모듬팩\n수량: 2', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 205, 'total_tokens': 226}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f33661fe-1c55-4082-9ca1-8b85003724a0-0')

In [13]:
create_order_parser.invoke(response)

OutputParserException: Invalid json output: 상품명: 개별 모듬팩
수량: 2

원래 {format_instructions} 포함 시키는 건가? 자동으로 이 부분 지정돼 있는 거 아니었음?

In [5]:
template = """
You are a robot designed to extract necessary parameters for processing each order request.
Only include the extracted parameters in your response.

Product Information:
{products}

User ID: {user_id}

User Input: {input}

Provide only the extracted parameters in the following format:
{format_instructions}
"""

from langchain_core.prompts import PromptTemplate

extract_order_args_prompt = PromptTemplate(
    template=template,
    input_variables=["user_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions()]},
)

In [37]:
extract_order_args_chain = extract_order_args_prompt | model

response = extract_order_args_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
response

'\n{\n    "user_id": 1,\n    "items": [\n        {\n            "product_name": "개별 모듬팩",\n            "quantity": 2,\n            "price": 13500\n        }\n    ]\n}'

In [39]:
print(response)


{
    "user_id": 1,
    "items": [
        {
            "product_name": "개별 모듬팩",
            "quantity": 2,
            "price": 13500
        }
    ]
}


In [40]:
create_order_parser.invoke(response)

CreateOrderData(user_id=1, items=[OrderItemData(product_name='개별 모듬팩', quantity=2, price=Decimal('13500'))])

format_instructions에 []로 여러 파서 담으면 결과도 [] 안에 담기는 건가?

In [42]:
extract_order_args_prompt = PromptTemplate(
    template=template,
    input_variables=["uesr_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions(),
                                               update_order_parser.get_format_instructions()]},
)

extract_order_args_chain = extract_order_args_prompt | model

response = extract_order_args_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
print(response)

[{'product_name': '개별 모듬팩', 'quantity': 2, 'price': 13500}]


In [22]:
create_order_parser.invoke(response.content)

OutputParserException: Failed to parse CreateOrderData from completion [{"user_id": 1, "items": [{"product_name": "\uac1c\ubcc4 \ubaa8\ub4ec\ud329", "quantity": 2, "price": 13500}]}]. Got: 1 validation error for CreateOrderData
  Input should be a valid dictionary or instance of CreateOrderData [type=model_type, input_value=[{'user_id': 1, 'items': ...': 2, 'price': 13500}]}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

In [25]:
response.content

'[{"user_id": 1, "items": [{"product_name": "개별 모듬팩", "quantity": 2, "price": 13500}]}]'

In [27]:
import json

response_list = json.loads(response.content)
response_list

[{'user_id': 1,
  'items': [{'product_name': '개별 모듬팩', 'quantity': 2, 'price': 13500}]}]

In [28]:
response_list[0]

{'user_id': 1,
 'items': [{'product_name': '개별 모듬팩', 'quantity': 2, 'price': 13500}]}

In [29]:

create_order_parser.invoke(response_list[0])

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [30]:
create_order_parser.parse(response_list[0])

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [31]:
str(response_list[0])

"{'user_id': 1, 'items': [{'product_name': '개별 모듬팩', 'quantity': 2, 'price': 13500}]}"

In [32]:

create_order_parser.invoke(str(response_list[0]))

OutputParserException: Invalid json output: {'user_id': 1, 'items': [{'product_name': '개별 모듬팩', 'quantity': 2, 'price': 13500}]}

# 랭체인 예제

In [33]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [34]:
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
output

'\n{\n    "setup": "Why did the tomato turn red?",\n    "punchline": "Because it saw the salad dressing!"\n}'

In [35]:
print(output)


{
    "setup": "Why did the tomato turn red?",
    "punchline": "Because it saw the salad dressing!"
}


# 파서 1개씩 사용하여 체인 구성

In [ ]:
from pydantic import BaseModel, Field, conint, condecimal
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class OrderItemData(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    quantity: conint(gt=0) = Field(..., description="Quantity of the product")
    price: condecimal(max_digits=10, decimal_places=2) = Field(..., description="Price of the product")

class CreateOrderData(BaseModel):
    user_id: int = Field(..., description="ID of the user creating the order")
    items: List[OrderItemData] = Field(..., description="List of items to order")

class UpdateOrderStatusData(BaseModel):
    order_id: int = Field(..., description="ID of the order to update")
    new_status: str = Field(..., description="New status of the order")

create_order_parser = PydanticOutputParser(pydantic_object=CreateOrderData)
update_order_parser = PydanticOutputParser(pydantic_object=UpdateOrderStatusData)

In [ ]:
template = """
You are a robot designed to extract necessary parameters for processing each order request.
Only include the extracted parameters in your response.

Product Information:
{products}

User ID: {user_id}

User Input: {input}

Provide only the extracted parameters in the following format:
{format_instructions}
"""

In [8]:
from langchain_core.prompts import PromptTemplate

extract_order_args_prompt = PromptTemplate(
    template=template,
    input_variables=["user_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions()]},
)
extract_order_args_chain = extract_order_args_prompt | model

response = extract_order_args_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
response

AIMessage(content='{\n  "user_id": 1,\n  "items": [\n    {\n      "product_name": "개별 모듬팩",\n      "quantity": 2,\n      "price": 13500\n    }\n  ]\n}', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 534, 'total_tokens': 585}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d659a2dd-7ecd-492f-a3c5-26a590fb4305-0')

In [9]:
extract_order_args_chain = extract_order_args_chain | create_order_parser

response = extract_order_args_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
response

CreateOrderData(user_id=1, items=[OrderItemData(product_name='개별 모듬팩', quantity=2, price=Decimal('13500'))])

In [43]:
extract_update_args_prompt = PromptTemplate(
    template=template,
    input_variables=["user_id", "input", "products"],
    partial_variables={"format_instructions": [update_order_parser.get_format_instructions()]},
)

extract_update_args_chain = extract_update_args_prompt | model

response = extract_update_args_chain.invoke(
    {"user_id": 1,
     "input": "3번 주문에서 개별 모듬팩 1개는 뺄게요",
     "products": products}
)
response

'\n{\n    "order_id": 3,\n    "new_status": "Cancelled"\n}'

: 

# 라우트

In [ ]:
def requeset_types_route(info):
    print("requeset_types_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경 요청" in info["request_type"].content.lower():
        return RunnableLambda(get_payment_completed_orders)
    elif "주문 취소 요청" in info["request_type"].content.lower():
        return RunnableLambda(get_changed_orders)
    else:
        return RunnableLambda(get_completed_orders)